In [1]:
import pandas as pd
import numpy as np
from pct.tree.heuristic.Heuristic import Heuristic5
from pct.tree.heuristic.NumericHeuristic import NumericHeuristic5
from pct.tree.splitter.splitter import Splitter
from pct.tree.tree import Tree
from pct.tree.ftest.ftest import FTest




## From YXH's data process

In [2]:

from sklearn.preprocessing import LabelEncoder

# Load the u.data dataset
u_data = pd.read_csv('ml-100k/u.data', sep='\t', header=None, names=['user_id', 'item_id', 'rating', 'timestamp'])

# Perform label encoding on user_id and item_id
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()

u_data['user_id'] = user_encoder.fit_transform(u_data['user_id'])
u_data['item_id'] = item_encoder.fit_transform(u_data['item_id'])

# Create the rating matrix
n_users = u_data['user_id'].nunique()
n_items = u_data['item_id'].nunique()

# Define the threshold
THRESHOLD = 3  # Ratings above this are "Lovers", below are "Haters"

# Create a user-item matrix with users as rows and items as columns, and fill missing values with 0
rating_matrix = u_data.pivot(index='user_id', columns='item_id', values='rating')
rating_matrix.index = u_data['user_id'].unique()  # Ensure user_id alignment
rating_matrix.columns = u_data['item_id'].unique()  # Ensure item_id alignment


# Apply threshold classification to the rating matrix (after thresholding)
rating_matrix_thresholded = np.where(rating_matrix > THRESHOLD, 1, np.where(rating_matrix > 0, -1, 0))

# Convert numpy array to pandas DataFrame for easier inspection
rating_matrix_thresholded_df = pd.DataFrame(rating_matrix_thresholded)


# Create rI and rU indexes for training set (R)
# rI: item to user ratings (dictionary of items with lists of user ratings)
rI = {}
for _, row in u_data.iterrows():
    item_id = row['item_id']
    user_id = row['user_id']
    rating = row['rating']
    if item_id not in rI:
        rI[item_id] = []
    rI[item_id].append((user_id, rating))

# rU: user to item ratings (dictionary of users with lists of item ratings)
rU = {}
for _, row in u_data.iterrows():
    user_id = row['user_id']
    item_id = row['item_id']
    rating = row['rating']
    if user_id not in rU:
        rU[user_id] = []
    rU[user_id].append((item_id, rating))

In [17]:
assert set(rI.keys()).issubset(set(rating_matrix.columns)), "Mismatch in rI keys!"
assert set(rU.keys()).issubset(set(rating_matrix.index)), "Mismatch in rU keys!"


In [12]:
# find and count users who have rated item 246 and their ratings
item_id = 55
print(f"Users who have rated item {item_id} and their ratings:")
print(rI[item_id])

# count users who have rated item 246 and their ratings
print(f"Number of users who have rated item {item_id}: {len(rI[item_id])}")




Users who have rated item 55 and their ratings:
[(np.int64(300), np.int64(4)), (np.int64(12), np.int64(5)), (np.int64(238), np.int64(4)), (np.int64(188), np.int64(5)), (np.int64(81), np.int64(3)), (np.int64(225), np.int64(4)), (np.int64(144), np.int64(5)), (np.int64(270), np.int64(3)), (np.int64(15), np.int64(5)), (np.int64(72), np.int64(4)), (np.int64(159), np.int64(5)), (np.int64(176), np.int64(5)), (np.int64(286), np.int64(5)), (np.int64(43), np.int64(2)), (np.int64(64), np.int64(3)), (np.int64(304), np.int64(1)), (np.int64(129), np.int64(5)), (np.int64(13), np.int64(5)), (np.int64(0), np.int64(4)), (np.int64(275), np.int64(5)), (np.int64(36), np.int64(5)), (np.int64(68), np.int64(5)), (np.int64(84), np.int64(4)), (np.int64(193), np.int64(5)), (np.int64(135), np.int64(4)), (np.int64(162), np.int64(4)), (np.int64(20), np.int64(5)), (np.int64(56), np.int64(3)), (np.int64(285), np.int64(2)), (np.int64(174), np.int64(2)), (np.int64(48), np.int64(5)), (np.int64(160), np.int64(3)), (np.in

In [13]:

print(f"Number of users who didn't rate item {item_id}:")
print(n_users - len(rI[item_id]))

Number of users who didn't rate item 55:
549


In [14]:
rating_matrix

item_id,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
user_id,,,,,,,,,,,,,,,,,,,,,
0,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
939,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# rating_matrix_thresholded_df.head()

In [18]:
# Optionally, print rI and rU to check the data structure for training and testing
print("\nSample rI (item to user ratings) for Training Set:")
for item in list(rI.keys())[:2]:  # Display sample item indices
    print(f"Item {item}: {rI[item]}")

print("\nSample rU (user to item ratings) for Training Set:")
for user in list(rU.keys())[:2]:  # Display sample user indices
    print(f"User {user}: {rU[user]}")


Sample rI (item to user ratings) for Training Set:
Item 241: [(np.int64(195), np.int64(3)), (np.int64(62), np.int64(3)), (np.int64(225), np.int64(5)), (np.int64(153), np.int64(3)), (np.int64(305), np.int64(5)), (np.int64(295), np.int64(4)), (np.int64(33), np.int64(5)), (np.int64(270), np.int64(4)), (np.int64(200), np.int64(4)), (np.int64(208), np.int64(4)), (np.int64(34), np.int64(2)), (np.int64(353), np.int64(5)), (np.int64(198), np.int64(5)), (np.int64(112), np.int64(2)), (np.int64(0), np.int64(5)), (np.int64(172), np.int64(5)), (np.int64(359), np.int64(4)), (np.int64(233), np.int64(4)), (np.int64(13), np.int64(4)), (np.int64(308), np.int64(4)), (np.int64(330), np.int64(4)), (np.int64(20), np.int64(3)), (np.int64(110), np.int64(4)), (np.int64(438), np.int64(5)), (np.int64(354), np.int64(4)), (np.int64(203), np.int64(5)), (np.int64(144), np.int64(5)), (np.int64(29), np.int64(5)), (np.int64(462), np.int64(2)), (np.int64(143), np.int64(4)), (np.int64(416), np.int64(3)), (np.int64(1), n

## initiate tree and splitter

In [3]:
tree = Tree(min_instances=6)
tree.fit(rating_matrix, u_data['rating'], target_weights=None, rI=rI, rU=rU)




✅ Converted x and y to DataFrame
✅ Assigned x and y
✅ Creating target weights...
✅ Identifying numerical and categorical attributes...
✅ Creating Splitter...
Initializing Splitter...
✅ Calling build()...
🌲 Building predictive clustering tree...


/Users/camillecu/Downloads/KUL/master_thesis/master_thesis_github/thesis_github/lib/python3.9/site-packages/numpy/_core/fromnumeric.py:84: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


🔍 Best item for splitting:  791
👥 Users who rated item 791: 86
❤️ Lovers: 59
💔 Haters: 27
❓ Unknowns: 857
🔄 Recursively building tree for subsets...
🌲 Building predictive clustering tree...


/Users/camillecu/Downloads/KUL/master_thesis/master_thesis_github/thesis_github/lib/python3.9/site-packages/numpy/_core/fromnumeric.py:84: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


🔍 Best item for splitting:  689
👥 Users who rated item 689: 155
❤️ Lovers: 84
💔 Haters: 71
❓ Unknowns: 52
🔄 Recursively building tree for subsets...
🌲 Building predictive clustering tree...
🔍 Best item for splitting:  None
🍃 Creating leaf node (no valid split found)...
🌲 Building predictive clustering tree...
🔍 Best item for splitting:  None
🍃 Creating leaf node (no valid split found)...
🌲 Building predictive clustering tree...


...

/Users/camillecu/Downloads/KUL/master_thesis/master_thesis_github/pct/tree/node/node.py:61: RuntimeWarning: invalid value encountered in divide
  prototype /= np.sum(
/Users/camillecu/Downloads/KUL/master_thesis/master_thesis_github/thesis_github/lib/python3.9/site-packages/numpy/_core/fromnumeric.py:84: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


🔍 Best item for splitting:  None
🍃 Creating leaf node (no valid split found)...
🌲 Building predictive clustering tree...
🔍 Best item for splitting:  1015
👥 Users who rated item 1015: 137
❤️ Lovers: 68
💔 Haters: 69
❓ Unknowns: 23
🔄 Recursively building tree for subsets...
🌲 Building predictive clustering tree...
🔍 Best item for splitting:  None
🍃 Creating leaf node (no valid split found)...
🌲 Building predictive clustering tree...


/Users/camillecu/Downloads/KUL/master_thesis/master_thesis_github/pct/tree/node/node.py:61: RuntimeWarning: invalid value encountered in divide
  prototype /= np.sum(
/Users/camillecu/Downloads/KUL/master_thesis/master_thesis_github/thesis_github/lib/python3.9/site-packages/numpy/_core/fromnumeric.py:84: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


🔍 Best item for splitting:  None
🍃 Creating leaf node (no valid split found)...
🌲 Building predictive clustering tree...


KeyboardInterrupt: 

In [7]:
# tree.print_tree_structure()